In [1]:
import pandas as pd
import numpy as np


filename = 'data/gxadmin_wf_conn'
# read raw file
df = pd.read_csv(filename, sep='\t', names=['wf_id', 'wf_updated', 'in_id', 'in_tool', 'in_tool_v', 'out_id', 'out_tool', 'out_tool_v', 'tool_inputs', 'tool_outputs', 'published'])
# remove last row
df = df[0:len(df) - 1]
rows = len(df)
# create tsv file
f_name = "data/workflow_connection_1911" + "_" + str(rows) + ".tsv"
df[:rows].to_csv(f_name, sep="\t", index=False)

In [112]:
df

,wf_id,wf_updated,in_id,in_tool,in_tool_v,out_id,out_tool,out_tool_v,tool_inputs,tool_outputs,published
0,5,2013-02-07 16:49:04.367628,27,Cut1,1.0.1,59,Paste1,1.0.0,input1,NaN,f
1,5,2013-02-07 16:49:04.367628,39,Filter1,1.1.0,46,Summary_Statistics1,1.1.0,input,NaN,f
2,5,2013-02-07 16:49:04.367628,49,Summary_Statistics1,1.1.0,32,Cut1,1.0.1,input,NaN,f
3,5,2013-02-07 16:49:04.367628,60,Paste1,1.0.0,63,Add_a_column1,1.1.0,input,NaN,f
4,5,2013-02-07 16:49:04.367628,62,Add_a_column1,1.1.0,38,addValue,1.0.0,input,NaN,f
...,...,...,...,...,...,...,...,...,...,...,...
320378,39075,2019-11-19 16:32:55.062139,592026,NaN,NaN,592033,toolshed.g2.bx.psu.edu/repos/iuc/raceid_inspec...,3.1,inputrds,output,f
320379,39075,2019-11-19 16:32:55.062139,592026,NaN,NaN,592029,toolshed.g2.bx.psu.edu/repos/iuc/raceid_inspec...,3.1,inputrds,output,f
320380,39077,2019-11-19 16:50:55.654888,592053,NaN,NaN,592058,toolshed.g2.bx.psu.edu/repos/iuc/gemini_load/g...,0.20.1+galaxy1,infile,NaN,f
320381,39084,2019-11-19 17:22:42.874235,592159,NaN,NaN,592169,toolshed.g2.bx.psu.edu/repos/lparsons/htseq_co...,0.9.1,gfffile,output,f


In [3]:
# read file
df_complete = pd.read_csv(f_name, sep='\t')
wf_id = list()
in_tool = list()
out_tool = list()
tool_input = list()
tool_output = list()
in_tool_v = list()
out_tool_v = list()

df_complete.replace(np.nan, "", inplace=True)

def format_tool_id(tool_id):
    t_split_ids = tool_id.split("/")
    if len(t_split_ids) > 1:
        id = t_split_ids[-2]
        version = t_split_ids[-1]
        return id, version
    else:
        return tool_id, ""

# format/clean data
size = len(df_complete)
for index, row in df_complete.iterrows():
    if index > size:
        break
    row['wf_id'] = str(row['wf_id']).strip()
    row['in_tool'] = str(row['in_tool']).strip()
    row['out_tool'] = str(row['out_tool']).strip()
    row['tool_inputs'] = str(row['tool_inputs']).strip()
    row['tool_outputs'] = str(row['tool_outputs']).strip()
    row['in_tool_v'] = str(row['in_tool_v']).strip()
    row['out_tool_v'] = str(row['out_tool_v']).strip()

    if len(row['in_tool']) > 0 and row['in_tool'] not in ["nan", np.nan] \
        and len(row['out_tool']) > 0 and row['out_tool'] not in ["nan", np.nan] \
        and len(row['tool_inputs']) > 0 and row['out_tool'] not in ["nan", np.nan] \
        and len(row['tool_outputs']) > 0 and row['out_tool'] not in ["nan", np.nan]:
        
        in_id, in_version = format_tool_id(row['in_tool'])
        out_id, out_version = format_tool_id(row['out_tool'])
        
        in_v = in_version
        out_v = out_version
        
        if in_v == "":
            in_v = row['in_tool_v']
        if out_v == "":
            out_v = row['out_tool_v']
        if len(in_v) > 0 and len(out_v) > 0:
            in_tool.append(in_id)
            out_tool.append(out_id)
            tool_input.append(row['tool_inputs'])
            tool_output.append(row['tool_outputs'])
            in_tool_v.append(in_v)
            out_tool_v.append(out_v)
            wf_id.append(row['wf_id'])
# save clean data as csv
corrected_data = {'wf_id': wf_id,
                  'in_tool': in_tool,
                  'in_tool_version': in_tool_v,
                  'in_tool_output': tool_output,
                  'out_tool': out_tool,
                  'out_tool_input': tool_input,
                  'out_tool_version': out_tool_v,
                 }
df_corrected_data = pd.DataFrame(corrected_data)

name = "data/corrected_workflow_connection_1911_" + str(len(in_tool)) + ".csv"
df_corrected_data.to_csv(name, sep=",", index=False)